In [1]:
import json
import tweepy


/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def load_token(path):
    with open(path, "r") as f:
        json_data = json.load(f)
        AKey = json_data["API Key"]
        ASec = json_data["API Secret Key"]
        AcToken = json_data["Access Token"]
        AcTokenSec = json_data["Access Token Secret"]
    return AKey, ASec, AcToken, AcTokenSec

In [3]:
path = "../../corpus/CorpusMakerAI.json"
AKey, ASec, AcToken, AcTokenSec = load_token(path)

In [4]:
class Tweet:
    def __init__(self, status):
        self.in_reply_to_status_id = status.in_reply_to_status_id
        self.text = status.text
        self.created_at = status.created_at
        self.screen_name = status.user.screen_name
        self.username = status.user.name
        self.user_id = status.user.id

In [5]:
def _in(arg1, arg2, mode="or"):
    result = False
    if isinstance(arg1, list):
        if mode == "and":
            for element in arg1:
                if isinstance(element, str):
                    if element not in arg2:
                        return False
            return True
        # or
        for element in arg1:
            if isinstance(element, str):
                if element in arg2:
                    result = True
                    break
            else:
                if element == arg2:
                    result = True
                    break
    else:
        if isinstance(arg1, str):
            if arg1 in arg2:
                result = True
        else:
            if arg1 == arg2:
                result = True
    return result

In [6]:
import spacy
import re
from datetime import timedelta
import os

class StreamListener(tweepy.StreamListener):

    def __init__(self, api, corpus_path, name):
        super(StreamListener, self).__init__()
        self.nlp =  spacy.load('ja_ginza')
        self.filter_pos = ["名詞", "動詞", "形状詞", "形容詞"]

        self.api = api
        self.lookup_ids = []
        self.reply_dict = dict()
        
        self.corpus_path = corpus_path
        self.name = name
        
        if name in os.listdir(corpus_path):
           
            with open(corpus_path+name, "r") as f:
                self.conversations = json.load(f)
            print("success load:", name) 
        else:
            self.conversations = {"convs":[]}

        self.counts = 0

    def on_status(self, status):
        self.counts += 1
        
        # if self.counts % 100 == 0:
        #     print("crawled...{0}, id_length: {1}".format(self.counts, len(self.lookup_ids)))

        # リプライではない -> NO
        if self.is_status_tweet(status):
            return
        
        # 無効ならば => NO
        if self.is_invalid_tweet(status):
            return
        
        self.lookup_ids.append(status.in_reply_to_status_id)
        self.reply_dict[status.in_reply_to_status_id] = Tweet(status)
        print(".", end='', flush=True)
        if len(self.lookup_ids) >= 50:
            print("\nCalling statuses_lookup API...")
            # 多分これで検索してる
            statuses = self.api.statuses_lookup(self.lookup_ids)

            for status in statuses:
                if self.is_status_tweet(status):
                    continue

                if self.is_invalid_tweet(status):
                    continue

                reply = self.reply_dict[status.id] 
                # リプライ先が同じユーザー？
                if status.user.id == reply.user_id:
                    continue
                
                self.add_conversation(status, reply)
                self.print_conversation(status, reply)
            self.lookup_ids = []
            self.reply_dict = {}

            

    def is_status_tweet(self, status):
        # リプライではないただのツイートか確認
        if status.in_reply_to_status_id is None:
            return True
    
    # いらないツイートか調べる
    def is_invalid_tweet(self, status):
        # print(status.lang )
        if status.lang != "ja":
            # 日本語か確認
            return True
        # print(status.text)
        if "bot" in status.user.screen_name:
            return True

        if re.search(r"https?://", status.text):
            return True

        if re.search(r"#(\w+)", status.text):
            # ハッシュタグ
            return True
        # print(status.text)
        # 複数の相手にリプライしているか？
        tweet = re.sub(r"@([A-Za-z0-9_]+)", "<unk>", status.text)
        if tweet.split().count("<unk>") > 1:
            return True

        # 長いツイートか？
        if len(tweet.replace("<unk>", "")) > 30:
            return True
        
        # 内容語が含まれているか
        if not self._is_contain_POS(status.text):
            return True
        
        # 発話の長さは適当か(形態素が6つ以上)


        return False
    
    def _is_contain_POS(self, text):
        doc = self.nlp(text, disable=['ner'])
        if len(doc) < 7:
            return False
        for token in doc:
            if "助動詞" not in token.tag_ and _in(self.filter_pos, token.tag_):
                return True
        return False
    
    # def _is_morpheme_len(self, text):

    
    def cleanup_text(self, status):
        text = re.sub(r"@([A-Za-z0-9_]+) ", "", status.text)
        text = re.sub("\s+", ' ', text).strip()
        return text.replace("&gt;", ">").replace("&lt;", "<").replace("&amp;", "&")
    

    def print_conversation(self, reply1, reply2):
        print('------------ 会話 ------------')
        print("reply1:@{}({}): {}".format(
            reply1.user.screen_name,
            reply1.created_at + timedelta(hours=+9),
            reply1.text)
        )
        print("reply2:@{}({}): {}".format(
            reply2.screen_name,
            reply2.created_at + timedelta(hours=+9),
            reply2.text)
        )
        print('-'*30)

    def add_conversation(self, reply1, reply2):
        reply1 = self.cleanup_text(reply1)
        reply2 = self.cleanup_text(reply2)
        data = {
            "rep1": reply1,
            "rep2": reply2
            }
        self.conversations["convs"].append(data)
    

    def save_conv(self):
        with open(self.corpus_path+self.name, "w") as f:
            json.dump(self.conversations, f, ensure_ascii=False, indent=4)


In [7]:
auth = tweepy.OAuthHandler(AKey, ASec)
auth.set_access_token(AcToken, AcTokenSec)
api = tweepy.API(auth)

In [8]:
out_path = "../../corpus/twitter/"
name = "conv.json"
listener = StreamListener(api, out_path, name)
streaming = tweepy.Stream(auth, listener)

listener._is_contain_POS("観光地の食べ物は高いですもんね")

success load: conv.json


True

In [9]:
while True:
    try:
        streaming.sample()
    except KeyboardInterrupt:
        streaming.disconnect()
        break
    except Exception as e:
        streaming.disconnect()
        print(e)

..................................................
Calling statuses_lookup API...
------------ 会話 ------------
reply1:@sho_chan_rnhm(2021-10-23 10:51:32): @azu_ntk あずちゃんの絵で
年賀状送って欲しい！
reply2:@azu_ntk(2021-10-23 15:13:46): @sho_chan_rnhm ぜひ送りたいところです‼️（？）
------------------------------
------------ 会話 ------------
reply1:@c09_y11(2021-10-23 14:53:49): @KEKKON_llll ありこまち！こまちはピチピチのJKでいるんだよ
reply2:@KEKKON_llll(2021-10-23 15:13:36): @c09_y11 鬱JDからいろいろ教わってるから大丈夫だよ！
------------------------------
------------ 会話 ------------
reply1:@_kunimitsu_mai(2021-10-23 15:10:42): @EmperorTemGS521 可愛いお写真だね☺️💕
真田くんも楽しそう🤭✨✨
reply2:@EmperorTemGS521(2021-10-23 15:14:00): @_kunimitsu_mai ありがとう🥺💕コースターお迎えしたのでプレゼントさせてね🍀
------------------------------
------------ 会話 ------------
reply1:@azup_alize(2021-10-23 15:10:16): @thepuroi 配信も？？？
reply2:@thepuroi(2021-10-23 15:13:28): @azup_alize さらばsweet step
------------------------------
------------ 会話 ------------
reply1:@ITSUKI34810(2021-10-23 15:10:41): @BOKITENSU あら

In [10]:
listener.conversations

{'convs': [{'rep1': 'ゲラッゲラッゲラッしか分からないから、勉強してきます！！笑', 'rep2': '古すぎたかあー😁'},
  {'rep1': '勇者組む予定だったんですね😓', 'rep2': '全部組みます(꒪˙꒳˙꒪ )👍'},
  {'rep1': 'キッズルーム広いなぁ。 今度いってみよーっと。', 'rep2': '広いけど何もないんよ🤣 でも快適！❤️'},
  {'rep1': 'さか というのお好きなように🙆🏻⸝⋆', 'rep2': 'さかちゃんて呼びますね🎶'},
  {'rep1': '明日もお仕事なのね\U0001f97a おやすみなさ〜い💤', 'rep2': 'おやすみなさーい'},
  {'rep1': '無事打てて何より✨今日明日はゆっくり休んでね～🌠💕', 'rep2': 'ありがとー！💕'},
  {'rep1': 'ちゃんと金曜日で安心😮\u200d💨爆笑', 'rep2': 'ほんとに！！ね！！！😌'},
  {'rep1': '最高にイージャン♪ですね', 'rep2': 'わぉ！イージャン☆ みんなラージャン友達～☆🦍🤝🦍'},
  {'rep1': 'よろしくお願いします🎶', 'rep2': 'よろしくお願いします〜！💕'},
  {'rep1': 'お疲れ様です よく頑張りましたね', 'rep2': 'やったぜ(◜◡‾)'},
  {'rep1': 'ﾟ+.*ｻｯｽｶﾞｧ━━━d(≧U≦●)━━━★*.+ﾟ', 'rep2': '(´>∀<｀)ゝ))ｴﾍﾍ'},
  {'rep1': '下手に攻めたら憶えられちゃう！', 'rep2': 'まるでアヌビス神じゃないですか！'},
  {'rep1': 'センサーを防水処理しときましょかwww', 'rep2': 'ライフジャケットの意味ねぇww'},
  {'rep1': '運動部文化部なんで減らないのがちなく', 'rep2': '自分も文化部（美術部）よ'},
  {'rep1': 'あんま いらないかな😂', 'rep2': '勝手にふえちゃうですよ😅'},
  {'rep1': 'まぢかよらっきー！！！', 'rep2': '答えれんかったら全然大丈夫ー！'},
  {'rep1': 'そうだといいですね…', 'rep2': '

In [115]:

listener.save_conv()

In [97]:
with open(out_path+"conv.json", "w") as f:
    json.dump(listener.conversations, f, ensure_ascii=False, indent=4)

In [98]:
listener._is_contain_POS("ありがとうございます😎")

True

In [99]:
nlp =  spacy.load('ja_ginza')


In [105]:
doc = nlp("ホニャラホニャ")
for token in doc:
    print(token, token.tag_)

ホニャラホニャ 名詞-普通名詞-一般
